This is practice and can be ignored.

In [1]:
# From pipeline textbook chapter, adapted for full accident control data
import pandas as pd


In [2]:

from sklearn.preprocessing import LabelEncoder
X_train = pd.read_csv("../Data_Cleaned/train_data/control_2class_X_train.csv")
y_train = pd.read_csv("../Data_Cleaned/train_data/control_2class_y_train.csv")
X_test = pd.read_csv("../Data_Cleaned/test_data/control_2class_X_test.csv")
y_test = pd.read_csv("../Data_Cleaned/test_data/control_2class_y_test.csv")

# Fix np formatting error
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

le = LabelEncoder()
y_train = le.fit_transform(y_train)

le.classes_
print(y_train)
print(X_train)


[0 0 0 ... 0 1 0]
            Street  Temperature(F)  Humidity(%)  Pressure(in)  Visibility(mi)  \
0          Highway            75.0         82.0         29.88            10.0   
1          Highway            45.0         89.0         29.44            10.0   
2       Interstate            91.0         66.0         29.95            10.0   
3            local            80.1         71.0         29.91            10.0   
4       Interstate            37.0         62.0         30.40            10.0   
...            ...             ...          ...           ...             ...   
40335  large_local            33.8         41.0         30.27            10.0   
40336  large_local            69.8         60.0         30.20            10.0   
40337   Interstate            43.0        100.0         29.74             2.0   
40338      Beltway            37.0         79.0         29.81            10.0   
40339        local            89.0         48.0         29.27            10.0   

      Win

In [9]:
import numpy as np
import scipy
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

param_range = scipy.stats.loguniform(0.0001, 1000.0)

np.random.seed(1)
param_range.rvs(10)

pipe_svc = make_pipeline(OneHotEncoder(handle_unknown='ignore'),
                         StandardScaler(with_mean = False),
                         PCA(n_components=10),
                         LinearSVC())

param_grid = [{'linearsvc__C': param_range},
              {'linearsvc__C': param_range}]

rs = RandomizedSearchCV(estimator=pipe_svc,
                        param_distributions=param_grid,
                        scoring='accuracy',
                        refit=True,
                        n_iter=20,
                        cv=10,
                        random_state=1,
                        n_jobs=1)

rs = rs.fit(X_train, y_train)
print(rs.best_score_)
print(rs.best_params_)

0.6602379771938522
{'linearsvc__C': np.float64(955.6385850092383)}


In [ ]:


# NEED TO FIGURE OUT ONE HOT ENCODER FOR PIPELINE, AND ADD IN FIRST
# Chose model based on https://scikit-learn.org/stable/machine_learning_map.html
# NEED TO ADD ARGS TO ALL THESE ASAP, AND CONFIRM I AM FOLLOWING HOW IT WORKS PROPERLY

pipe = make_pipeline(OneHotEncoder(handle_unknown='ignore'),
                     StandardScaler(with_mean=False),
                     PCA(n_components=10),
                     LinearSVC())

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
test_acc = pipe.score(X_test, y_test)
print(f"Test Accuracy: {test_acc:.3f}")

Test Accuracy: 0.673


In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.model_selection import StratifiedKFold

scores = cross_val_score(estimator = pipe_lr,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         n_jobs = 1)
print(f'CV Accuracy scores: {scores}')
print(f'CV Accuracy: {np.mean(scores):.3f} '
      f'+/- {np.std(scores):.3f}')

CV Accuracy scores: [0.78606295 0.78619529 0.78619529 0.78619529 0.78619529 0.78619529
 0.78619529 0.78619529 0.78602694 0.78602694]
CV Accuracy: 0.786 +/- 0.000


In [6]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits = 10).split(X_train, y_train)
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print(f'Fold: {k+1:02d}, '
          f'Class distr.: {np.bincount(y_train[train])}, '
          f'Acc.: {score:.3f}')
mean_acc = np.mean(scores)
std_acc = np.std(scores)
print(f'\nCV accuracy: {mean_acc:.3f} +/- {std_acc:.3f}')

Fold: 01, Class distr.: [  157 42028  4808  6467], Acc.: 0.786
Fold: 02, Class distr.: [  158 42028  4808  6467], Acc.: 0.786
Fold: 03, Class distr.: [  158 42028  4808  6467], Acc.: 0.786
Fold: 04, Class distr.: [  158 42028  4808  6467], Acc.: 0.786
Fold: 05, Class distr.: [  158 42028  4807  6468], Acc.: 0.786
Fold: 06, Class distr.: [  158 42028  4807  6468], Acc.: 0.786
Fold: 07, Class distr.: [  157 42028  4808  6468], Acc.: 0.786
Fold: 08, Class distr.: [  157 42028  4808  6468], Acc.: 0.786
Fold: 09, Class distr.: [  157 42029  4808  6467], Acc.: 0.786
Fold: 10, Class distr.: [  157 42029  4808  6467], Acc.: 0.786

CV accuracy: 0.786 +/- 0.000
